In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [3]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [4]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_5436\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [27]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/Vissy-Women’s-Sandals-Kasut-Wanita-Shoe-Shoes-Fashion-Travel-Lady-0018-LS-–-3-Colors-i.12216743.19643242404?sp_atk=9a56575d-3f82-4d9a-83ab-d919b72edc30&xptdk=9a56575d-3f82-4d9a-83ab-d919b72edc30
/Atikota-Women-Korean-Sneakers-New-Dad-Shoes-Increase-Comfortable-Small-White-Shoes-for-Ladies-Students-i.261956494.19055875227?sp_atk=cec629dc-2ec2-4e9e-8a72-56e72cb33988&xptdk=cec629dc-2ec2-4e9e-8a72-56e72cb33988
/SUBEI-1PC-Lady-Shoe-Clip-High-Heel-Charm-Buckle-Shiny-Decorative-Clips-Women-Bride-Wedding-Square-Clamp-Rhinestone-Shoe-Decorations-i.70512536.6286251658?sp_atk=0cba5c62-1db4-401c-940d-7399cdcd9f9e&xptdk=0cba5c62-1db4-401c-940d-7399cdcd9f9e
/Fashion-Women-Midi-Heels-Sandals-S24-i.11885141.19431308140?sp_atk=2b576b90-14d5-44ca-8c4b-86f3fe86c073&xptdk=2b576b90-14d5-44ca-8c4b-86f3fe86c073
/K6-READY-STOCK-MYFOOYIN-women's-shoes-i.24484132.3432122214?sp_atk=3402b31f-e985-4c6f-aefd-76c7ec88163d&xptdk=3402b31f-e985-4c6f-aefd-76c7ec88163d
/Women-Wedge-Sandals-Retro-Light-Carved-Hollow-Open

In [28]:
len(the_links)

60

In [29]:
items_list=[]
reviews_list=[]

In [30]:
for i in range (0,len(the_links)):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, description, image))

    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        
        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
0
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
1
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
2
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
3
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
4
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, 

In [31]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [32]:
items_df.shape

(60, 10)

In [33]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,ca28bb24-ca5c-4de1-a6b8-dbb824f05980,Vissy Women’s Sandals Kasut Wanita Shoe Shoes ...,4.8,298,893,Vissy,RM7.90,https://shopee.com.my/Vissy-Women’s-Sandals-Ka...,[Height = 1.5cm (Approx.)],https://down-my.img.susercontent.com/file/31d7...
1,b94dcdc4-40e7-43ac-8d40-58c1ffd6d67b,Atikota Women Korean Sneakers New Dad Shoes In...,4.8,909,2.3k,Atikota Fashion Shop,RM25.86 - RM29.86,https://shopee.com.my/Atikota-Women-Korean-Sne...,[Size: 35 36 37 38 39 40 41],https://down-my.img.susercontent.com/file/sg-1...
2,bba588e8-e0e9-4906-93db-61f6279d39b3,SUBEI 1PC Lady Shoe Clip High Heel Charm Buckl...,4.8,3.7k,19.4k,necklace&earrings,RM3.32 - RM8.67,https://shopee.com.my/SUBEI-1PC-Lady-Shoe-Clip...,"[Use for: Bride, Bridal Shoes\nSpecial design ...",https://down-my.img.susercontent.com/file/ead5...
3,5ad28ef1-59de-46be-84b0-c81e668cc3c0,Fashion Women Midi Heels Sandals S24,4.8,528,1.6k,VIVI MALL,RM15.90 - RM16.90,https://shopee.com.my/Fashion-Women-Midi-Heels...,[Fashion Women Midi Heels Sandals S24],https://down-my.img.susercontent.com/file/b16b...
4,ec703377-f710-422e-b88a-5c67ef85b992,K6 READY STOCK MYFOOYIN women's shoes,4.9,18k,43.4k,Myfooyin,RM4.90,https://shopee.com.my/K6-READY-STOCK-MYFOOYIN-...,[Size 36 : Foot Length 23.0 cm \nSize 37 : Foo...,https://down-my.img.susercontent.com/file/95d7...


In [34]:
reviews_df.shape

(4811, 6)

In [35]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
4806,c35fd40c-e781-4541-9290-0043068ff56c,e3de1dc4-965e-4b57-964a-0078d86e7ffb,2023-07-16 18:16,"Variation: Black,40",2,None
4807,8be3b2bc-02dc-4b84-ad4c-dbec37809255,e3de1dc4-965e-4b57-964a-0078d86e7ffb,2023-10-10 11:37,"Variation: Black,39",1,Comfortability:size was too small. it fits me ...
4808,2f772d23-a7cb-4f73-b5e7-552f607a8703,e3de1dc4-965e-4b57-964a-0078d86e7ffb,2023-01-06 11:00,"Variation: Black,43/44 (Remark Size)",1,None
4809,3419b5cb-4e99-411d-9380-43e3830f4031,e3de1dc4-965e-4b57-964a-0078d86e7ffb,2023-08-15 15:52,"Variation: White,42",1,boleh la
4810,a0c247fd-a7a6-4d9d-8f76-bd802351a11d,e3de1dc4-965e-4b57-964a-0078d86e7ffb,2023-06-07 12:58,"Variation: Black,41",1,None


In [36]:
reviews_df.to_json('womenshoes_reviews_3.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('womenshoes_items_3.json', orient='records', lines=True, force_ascii=False, default_handler=str)